# Experiments on backward variational ICA 

***Uncomment and run the following cell if you're using Collab***

In [1]:
# !rm -rf *
# !git clone https://github.com/mchagneux/backward_ica.git
# !mv backward_ica/* ./
# !rm -rf backward_ica

### Imports

In [2]:
from src.elbo import linear_gaussian_elbo
from src.hmm import LinearGaussianHMM
from src import kalman
from jax.random import PRNGKey
import jax.numpy as jnp
import jax
import optax
from jax import random
from src.misc import *
jax.config.update("jax_enable_x64", True)
# jax.config.update("jax_debug_nans", True)
key = PRNGKey(0)

## Introduction

This notebook is comprised of a series of experiments that attempt to recover expectations $\mathbb{E}[h(z_{1:t})|x_{1:t}]$ via variational approximations, when the process $(z_t, x_t)_{t \ge 1}$ is an HMM. The main metric $\ell$ all along is the MSE against the true states when $h$ is a plain sum, ie

$$\ell = \left(\sum_{t=1}^T z_t^* - \sum_{t=1}^T \mathbb{E}_{q_T(z_t)}[z_t] \right)^2$$

where $q_T(z_t) = q(z_t|x_{1:T})$ is the marginal smoothing distribution at $t$.

In all the following, we assume that the variational smoothing distribution factorizes as $q_\phi(z_{1:t}|x_{1:t}) = q_\phi(z_t|x_{1:t}) \prod_{s=1}^{t-1} q_\phi(z_s|z_{s+1},x_{1:s})$. We always assume that $$q_\phi(z_t|x_{1:t}) \sim \mathcal{N}(\mu_{1:t}, \Sigma_{1:t})$$ and 

$$q_\phi(z_s|z_{s+1},x_{1:s}) \sim \mathcal{N}(\overleftarrow{\mu}_{1:t}(z_{s+1}), \overleftarrow{\Sigma}_{1:t})$$

In the following, we make several assumptions on both $p_\theta$ and $q_\phi$.


In this case, not only should the expectations be correctly recovered, but parameters in $\phi$ and $\theta$ should be identifiable. We also know that in this case the best estimate of $z_{1:t}^*$ for any sequence is obtained via the Kalman smoothing recursions applied with parameters $\theta$ on the observations $x_{1:t}$. 



## 1. Linear Gaussian HMM 

First we assume that observation sequences $x_{1:T}$ arise from $p_\theta(z_{1:t},x_{1:t})$ defined as
$$z_t = A_\theta z_{t-1} + a_\theta + \eta_\theta$$ 
$$x_t = B_\theta z_t + b_\theta + \epsilon_\theta$$

where $\eta_\theta \sim \mathcal{N}(0,Q_\theta)$ and $\epsilon_\theta \sim \mathcal{N}(0,R_\theta)$

### 1. a. Approximated by a linear Gaussian HMM

We start by recovering $p_\theta$ when $q_\phi$ is in the family of the true p. We do this by prescribing the p for $q_\phi$ in forward time with a similar HMM structure as $p_\theta$ (but random initial parameters), and in this case the parameters of the filtering backward distributions exist via Kalman recursions and closed-form definitions.

In [3]:
state_dim, obs_dim = 2, 2 
num_sequences = 20
length = 8

key, *subkeys = random.split(key,3)

p_raw = LinearGaussianHMM.get_random_model(key=subkeys[0], state_dim=state_dim, obs_dim=obs_dim)
q_raw = LinearGaussianHMM.get_random_model(key=subkeys[1], state_dim=state_dim, obs_dim=obs_dim)

p = actual_model_from_raw_parameters(p_raw)
q = actual_model_from_raw_parameters(q_raw)

linear_gaussian_sampler = jax.vmap(LinearGaussianHMM.sample_joint_sequence, in_axes=(0, None, None))
key, *subkeys = random.split(key, num_sequences+1)
state_sequences, obs_sequences = linear_gaussian_sampler(jnp.array(subkeys), p, length)


filter_obs_sequences = jax.vmap(kalman.filter, in_axes=(0, None))
elbo_sequences = jax.vmap(linear_gaussian_elbo, in_axes=(None, None, 0))

average_evidence_across_sequences = jnp.mean(filter_obs_sequences(obs_sequences, p)[-1])
average_elbo_across_sequences_with_true_model = jnp.mean(elbo_sequences(p_raw, p_raw, obs_sequences))
print('Difference mean evidence Kalman and mean ELBO when q=p:', jnp.abs(average_evidence_across_sequences-average_elbo_across_sequences_with_true_model))
average_elbo_across_sequences_with_init_q = jnp.mean(elbo_sequences(p_raw, q_raw, obs_sequences))
print('Different mean evidence and mean ELBO when q=q0:', jnp.abs(average_evidence_across_sequences-average_elbo_across_sequences_with_init_q))


optimizer = optax.adam(learning_rate=-1e-3)

loss = lambda p_raw, q_raw, observations: linear_gaussian_elbo(p_raw, q_raw, observations)

def fit(p_raw, q_raw, optimizer: optax.GradientTransformation) -> optax.Params:
    opt_state = optimizer.init(q_raw)

    @jax.jit
    def step(p_raw, q_raw, opt_state, batch):
        loss_value, grads = jax.value_and_grad(loss, argnums=1)(p_raw, q_raw, batch)
        updates, opt_state = optimizer.update(grads, opt_state, q_raw)
        q_raw = optax.apply_updates(q_raw, updates)
        return p_raw, q_raw, opt_state, -loss_value

    eps = jnp.inf
    old_mean_epoch_loss = -average_elbo_across_sequences_with_init_q
    epoch_nb = 0
    while eps > 1e-2:
        epoch_loss = 0.0
        for batch in obs_sequences: 
            p_raw, q_raw, opt_state, loss_value = step(p_raw, q_raw, opt_state, batch)
            epoch_loss += loss_value
        mean_epoch_loss = epoch_loss/len(obs_sequences)
        eps = jnp.abs(mean_epoch_loss - old_mean_epoch_loss)
        if epoch_nb % 10 == 0: print('Change in average elbo from last iteration:', eps)
        epoch_nb+=1
        old_mean_epoch_loss = mean_epoch_loss
    return q_raw 


fitted_q_raw = fit(p_raw, q_raw, optimizer)

Difference mean evidence Kalman and mean ELBO when q=p: 2.842170943040401e-14
Different mean evidence and mean ELBO when q=q0: 63926.53136294946
Change in average elbo from last iteration: 36966.195978419404
Change in average elbo from last iteration: 361.8131073848094
Change in average elbo from last iteration: 31.633543857033487
Change in average elbo from last iteration: 2.264612917233592
Change in average elbo from last iteration: 4.431943899362466
Change in average elbo from last iteration: 4.236690914804512
Change in average elbo from last iteration: 3.8156304158587773
Change in average elbo from last iteration: 3.4448717667738435
Change in average elbo from last iteration: 2.923155833980985
Change in average elbo from last iteration: 2.831830018943066
Change in average elbo from last iteration: 2.4411804673708843
Change in average elbo from last iteration: 2.358355181386372
Change in average elbo from last iteration: 1.9797915903304073
Change in average elbo from last iteration:

In [4]:
### need to be changed to jax version 
# # checking expectations under approximate p when the additive functional is just the sum 
# with torch.no_grad():
#     additive_functional = partial(torch.sum, dim=0)
#     smoothed_with_true_model = mse_expectation_against_true_states(state_sequences, observation_sequences, hmm.model, additive_functional)
#     smoothed_with_approximate_model = mse_expectation_against_true_states(state_sequences, observation_sequences, q, additive_functional)

#     print('MSE when smoothed with p:',smoothed_with_true_model)
#     print('MSE when smoothed with q:',smoothed_with_approximate_model)

### 1. b. Using a neural network to compute the backward parameters instead of Kalman recursions
We make the same assumptions on $p_\theta$ but now we attempt to recover the backward parameters via neural network.

## 2. A nonlinear emission p

We now assume that $p_\theta$ has a nonlinear emission distribution, ie. $x_t  = f_\theta(z_t) + \epsilon$.

### 2. a. Approximated by a linear Gaussian p.
We keep a linear gaussian distribution for $q_\phi$, but we add a mapping to compute the expectation of the emission term from $p_\theta$. We need to approximate the following quantity:

$$\mathbb{E}_{q(z_t|z_{t+1}, x_{1:t})}\left[(x_t - f_\theta(z_t))^T R^{{\theta}^{-1}}(x_t - f_\theta(z_t))\right]$$

And similarly for the last expectation under the filtering distribution: 

$$\mathbb{E}_{q(z_T|x_{1:T})}\left[(x_T - f_\theta(z_T))^T R^{{\theta}^{-1}}(x_T - f_\theta(z_T))\right]$$

#### 2. a. i. A sampling-free approach. 


If we know the expectation $\mu$ and variance $\Sigma$ of a random variable $v$ (which need not be Gaussian):

$$\mathbb{E}_{v}\left[(x - v)^T \Omega (x - v)\right] = tr(\Sigma \Omega) + (\mu - x)^T \Omega (\mu - x)$$

Suppose we a have neural network which approximates the mean and variance of $v \sim f_\theta(z)$ when $z \sim p_z$, given parameters of $p_z$. Denote $\tilde{\mu}$ and $\tilde{\Sigma}$ these means and variances estimated by this network. For the filtering case, we feed the network with filtering mean and covariance at $T$ to obtain an estimate of $\tilde{\mu}$ and $\tilde{\Sigma}$, then:

$$\mathbb{E}_{q(z_T|x_{1:T})}\left[(x_T - f_\theta(z_T))^T R^{{\theta}^{-1}}(x_T - f_\theta(z_T))\right] = tr(\tilde{\Sigma} \Omega) + (\tilde{\mu} - x)^T R^{{\theta}^{-1}} (\tilde{\mu} - x)$$

For the backwards case this is not as simple, because: $\overleftarrow{\mu}_{1:t}$ is a function of $z_{t+1}$, therefore $\mathbb{E}_{q(z_t|z_{t+1}, x_{1:t})}[f_\theta(z_t)]$ and $\mathbb{V}_{q(z_t|z_{t+1}, x_{1:t})}[f_\theta(z_t)]$ are also functions of $z_{t+1}$. 

We still attempt to use one network for both the fitlering and the backwards via the following scheme: 

- Build a neural network $g_\alpha(A, a, \Sigma)$ which outputs $\tilde{A}, \tilde{a}$ and $\tilde{\Sigma}$
- For the backwards case, use $A = \overleftarrow{A}_{1:t}, a = \overleftarrow{a}_{1:t}$ and $\Sigma = \overleftarrow{\Sigma}_{1:t}$, and consider that $\tilde{\mu} = \tilde{A}z_{t+1} + \tilde{a}$, while $\tilde{\Sigma}$ does not depend on $z_{t+1}$ (which is knowingly false). In this case, the quadratic form build for $\tilde{A}$ and $\tilde{a}$ is a quadratic form in $z_{t+1}$ as wanted.
- For the backwards case, use $A = 0, a = a_{1:t}$ and $\Sigma = \Sigma_{1:t}$, and consider that $\tilde{\mu} = \tilde{a}$ (without using the output $\tilde{A}$).

*This method, which is tried below: fails to learn anything as of now.*

#### 2. a. i. The Johnson trick
